In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = os.environ.get("SCHEDVIEW_VISIT_ORIGIN", "lsstcam")
day_obs = int(
    os.environ.get(
        "SCHEDVIEW_DAY_OBS",
        (datetime.date.today() - datetime.timedelta(days=1)).isoformat(),
    ).replace("-", "")
)
extrap_day_obs = int(
    os.environ.get(
        "EXTRAP_DAY_OBS",
        "20261101"
    )
)

In [ ]:
"""
# To render this notebook to html, in the current directory with the current python environment

export ACCESS_TOKEN_FILE=${HOME}/.lsst/usdf_access_token
export SCHEDVIEW_VISIT_ORIGIN=lsstcam
export SCHEDVIEW_DAY_OBS='20260130'
export EXTRAP_DAY_OBS='20261101'
time jupyter nbconvert \
    --to html \
    --execute \
    --no-input \
    --ExecutePreprocessor.kernel_name=python3 \
    --ExecutePreprocessor.startup_timeout=3600 \
    --ExecutePreprocessor.timeout=3600 \
    pre-progress.ipynb

# The public USDF page served at
#    https://s3df.slac.stanford.edu/data/rubin/sim-data/schedview/reports/preprogress/lsstcam/2026/01/30/pre-progress_2026-01-30.html
# is found at
#    /sdf/group/rubin/web_data/sim-data/schedview/reports/preprogress/lsstcam/2026/01/30/pre-progress_2026-01-30.html

# You can set up the conda environment used to automatically generate these pages with
source /sdf/group/rubin/sw/w_latest/loadLSST.sh
conda activate /sdf/data/rubin/shared/scheduler/envs/prenight
"""
pass

In [ ]:
import os
import sys

#sched_source = 'env'
#sched_source = 'shared'
sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.4.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.22.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.6.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.22.0a2")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_nights-0.11.0")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import calendar
import datetime
import os
import sqlite3
import sys
from collections import defaultdict, OrderedDict
from pathlib import Path
from collections import ChainMap
from tempfile import TemporaryDirectory

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.maf
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.collect.visits
import schedview.collect.sv
import rubin_sim.sim_archive
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS
from rubin_nights.reference_values import SCIENCE_PROGRAMS
from schedview.plot.survey_skyproj import map_hpix_in_laea_and_mcbryde, map_metric_in_laea_and_mcbryde
from schedview.plot import mpl_fig_to_html

In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(no_sky=True)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
science_programs_to_include = SCIENCE_PROGRAMS
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

In [ ]:
extrap_day_obs = DayObs.from_date(extrap_day_obs)

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)
survey_visits = visits.loc[visits['science_program'].isin(science_programs_to_include), :] if visit_origin == 'lsstcam' else visits

In [ ]:
baseline_visits = schedview.collect.visits.read_visits(extrap_day_obs, 'baseline',  stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=4000)

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)

# Pre-start Survey Progress

## Progress as measured by scalar ("summary") metrics

**Any MAF summary metric (such as those reported [here](https://usdf-maf.slac.stanford.edu/summaryStats?runId=1)) can potentially be computed and reported on either of these plots, up to any cutoff date,** such as the cutoff for inclusion in DR1, or the end of the 10 year survey.

Two examples are shown below:
 - the coadd depth in i, which shows the advantages plotting the instantaneous metric value vs. date.
 - the total area with more than 50 visits in square degrees (f0Area), an example for which the extrapolated final value is a more useful measure of progress toward the final objective.

In the examples here, metrics are extrapolated to 2026-11-01.

In [ ]:
BENCHMARK_AREA = 18000
MIN_N_VISITS = 50
NSIDE = 32
BAND = 'i'

In [ ]:
def make_median_depth_metric_bundle(run_name, max_mjd=99999, nside=NSIDE, band=BAND):
    slicer = maf.HealpixSlicer(nside=nside, verbose=False)
    metric = maf.Coaddm5Metric()
    summary_metrics = [maf.MedianMetric(metric_name=f'median_depth_{band}')]
    bundle = maf.MetricBundle(
        metric,
        slicer,
        constraint=f'band == "{band}" AND observationStartMJD <= {max_mjd} and fiveSigmaDepth is not NULL',
        summary_metrics=summary_metrics,
        run_name=run_name,
    )
    return bundle


In [ ]:
def make_fOAreaMin_metric_bundle(run_name, max_mjd=99999, nside=NSIDE):
    slicer = maf.HealpixSlicer(nside=nside, verbose=False)
    metric = maf.CountExplimMetric(metric_name="fO")
    summary_metrics = [
        maf.FOArea(
            nside=slicer.nside,
            norm=False,
            metric_name=f"fOArea{MIN_N_VISITS}",
            asky=BENCHMARK_AREA,
            n_visit=MIN_N_VISITS,
        )
    ]
    bundle = maf.MetricBundle(
        metric,
        slicer,
        constraint=f'observationStartMJD <= {max_mjd}',
        summary_metrics=summary_metrics,
        run_name=run_name,
    )
    return bundle


In [ ]:
def extrapolate_metrics_from_mjd(start_opsimdf, end_opsimdf, mjd):

    with TemporaryDirectory() as chimera_dir:
        chimera_dirpath = Path(chimera_dir)
        chimera_fname = chimera_dirpath.joinpath('chimera.db').as_posix()
        maf_dirpath = chimera_dirpath.joinpath('maf')
        maf_dirpath.mkdir()
        
        chimera_opsimdf = pd.concat((
            start_opsimdf.loc[start_opsimdf.observationStartMJD <= mjd, :], 
            end_opsimdf.loc[end_opsimdf.observationStartMJD > mjd, :]
        ))
        with sqlite3.connect(chimera_fname) as connection:
            chimera_opsimdf.to_sql("observations", connection, index=False, if_exists='replace')
        
        run_name = f'mjd_{mjd}'
        bundles = {
            f'median_depth_{BAND}': make_median_depth_metric_bundle(run_name),
            f'fOArea{MIN_N_VISITS}': make_fOAreaMin_metric_bundle(run_name)
        }
        bundle_group = maf.MetricBundleGroup(
            bundles, chimera_fname, out_dir=maf_dirpath.as_posix()
        )

        bundle_group.run_all()
        bundle_group.summary_all()
        summary_values = dict(ChainMap(*[bundles[b].summary_values for b in bundles]))

    return summary_values

In [ ]:
# MS = month start, YS = year start ( https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases )
#freq = 'MS'
# freq = 'YS'
completed_freq = 'W'

start_datetime = Time(baseline_visits['observationStartMJD'].min(), format='mjd').datetime
current_datetime = Time(survey_visits['observationStartMJD'].max(), format='mjd').datetime 
if completed_freq == "MS":
    cutoff_datetime = current_datetime + datetime.timedelta(days=31)
elif completed_freq == "YS":
    cutoff_datetime = current_datetime + datetime.timedelta(days=366)
elif completed_freq == 'W':
    cutoff_datetime = current_datetime + datetime.timedelta(days=7)
else:
    assert False, completed_freq + " not recognized"

timestamps = pd.date_range(start=f'{start_datetime.year}-{start_datetime.month}-01', end=f'{cutoff_datetime.year}-{cutoff_datetime.month}-01', freq=completed_freq)
completed_mjds = (timestamps.to_julian_date().values - 2400000.5).astype(int)

end_datetime = Time(baseline_visits['observationStartMJD'].max(), format='mjd').datetime
future_freq = 'MS'
if future_freq == "MS":
    end_datetime = end_datetime + datetime.timedelta(days=31)
elif completed_freq == "YS":
    end_datetime = end_datetime + datetime.timedelta(days=366)
elif completed_freq == 'W':
    end_datetime = end_datetime + datetime.timedelta(days=7)
else:
    assert False, completed_freq + " not recognized"

future_timestamps = pd.date_range(start=f'{cutoff_datetime.year}-{cutoff_datetime.month}-01', end=f'{end_datetime.year}-{end_datetime.month}-01', freq=future_freq)
future_mjds = (future_timestamps.to_julian_date().values - 2400000.5).astype(int)
mjds = np.sort(np.unique(np.concatenate([completed_mjds, future_mjds])))

In [ ]:
extrapolated_summary_dict = {}
for mjd in completed_mjds:
    try:
        extrapolated_summary_dict[mjd] = extrapolate_metrics_from_mjd(survey_visits, baseline_visits, mjd)
    except:
        pass

extrapolated_summary = pd.DataFrame(extrapolated_summary_dict).T
extrapolated_summary.index.name = 'mjd'
extrapolated_summary.columns = pd.MultiIndex.from_product([['extrapolated'], extrapolated_summary.columns])

In [ ]:
def compute_metrics_at_mjd(opsim_fname, mjd):
    with TemporaryDirectory() as maf_dir:
        maf_dirpath = Path(maf_dir)

        run_name = f'mjd_{mjd}'
        bundles = {
            f'median_depth_{BAND}': make_median_depth_metric_bundle(run_name, mjd),
            f'fOArea{MIN_N_VISITS}': make_fOAreaMin_metric_bundle(run_name, mjd)
        }
        bundle_group = maf.MetricBundleGroup(
            bundles, opsim_fname, out_dir=maf_dirpath.as_posix()
        )

        bundle_group.run_all()
        bundle_group.summary_all()
        summary_values = dict(ChainMap(*[bundles[b].summary_values for b in bundles]))

    return summary_values

In [ ]:
with TemporaryDirectory() as data_dir:
    data_dirpath = Path(data_dir)
    completed_fname = data_dirpath.joinpath('completed.db').as_posix()
    maf_dirpath = data_dirpath.joinpath('maf')
    maf_dirpath.mkdir()

    with sqlite3.connect(completed_fname) as connection:
        survey_visits.to_sql("observations", connection, index=False, if_exists='replace')
    
    snapshot_summary_dict = {}
    for mjd in completed_mjds[1:]:
        snapshot_summary_dict[mjd] = compute_metrics_at_mjd(completed_fname, mjd)

snapshot_summary = pd.DataFrame(snapshot_summary_dict).T
snapshot_summary.index.name = 'mjd'
snapshot_summary.columns = pd.MultiIndex.from_product([['snapshot'], snapshot_summary.columns])

In [ ]:
baseline_snapshot_summary = pd.DataFrame({mjd: compute_metrics_at_mjd(get_baseline(), mjd) for mjd in mjds[1:]}).T
baseline_snapshot_summary.index.name = 'mjd'
baseline_snapshot_summary.columns = pd.MultiIndex.from_product([['baseline'], baseline_snapshot_summary.columns])

In [ ]:
summary = extrapolated_summary.merge(snapshot_summary, how='outer', left_index=True, right_index=True).merge(baseline_snapshot_summary, how='outer', left_index=True, right_index=True)
summary.columns.names = ['visits', 'metric']
summary.columns = summary.columns.reorder_levels(['metric', 'visits'])

In [ ]:
summary['date'] = pd.to_datetime(2400000.5 + summary.index, unit='D', origin='julian')

In [ ]:
median_depth = summary[f'median_depth_{BAND}']
median_depth['date'] = summary['date']
median_depth_ds = bokeh.models.ColumnDataSource(median_depth)

metric_at_date_fig = bokeh.plotting.figure(title=f"Median {BAND} depth at date", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
metric_at_date_fig.line(x='date', y='baseline', color='lightgreen', width=5, source=median_depth_ds)
metric_at_date_fig.line(x='date', y='snapshot', color='black', source=median_depth_ds)
metric_at_date_fig.scatter(x='date', y='snapshot', color='black', source=median_depth_ds)

baseline_final_depth = median_depth.loc[median_depth.index.max(), 'baseline']
extrapolated_metric_fig = bokeh.plotting.figure(title=f"Final {BAND} depth extrapolated from date to end with baseline", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
extrapolated_metric_fig.ray(x=median_depth['date'].min(), y=baseline_final_depth, length=0, angle=0, width=5, color='lightgreen')
extrapolated_metric_fig.line(x='date', y='extrapolated', color='black', source=median_depth_ds)
extrapolated_metric_fig.scatter(x='date', y='extrapolated', color='black', source=median_depth_ds)

metric_figs = bokeh.layouts.row(metric_at_date_fig, extrapolated_metric_fig)
bokeh.io.show(metric_figs)

In [ ]:
fOArea = summary[f'fOArea{MIN_N_VISITS}']
fOArea.loc[fOArea.baseline < 0, 'baseline'] = 0
fOArea['date'] = summary['date']
fOArea_ds = bokeh.models.ColumnDataSource(fOArea)

metric_at_date_fig = bokeh.plotting.figure(title=f"Square degrees with more than {MIN_N_VISITS} visits at data", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
metric_at_date_fig.line(x='date', y='baseline', color='lightgreen', width=5, source=fOArea_ds)
metric_at_date_fig.line(x='date', y='snapshot', color='black', source=fOArea_ds)
metric_at_date_fig.scatter(x='date', y='snapshot', color='black', source=fOArea_ds)

baseline_final_fOArea = fOArea.loc[fOArea.index.max(), 'baseline']
extrapolated_metric_fig = bokeh.plotting.figure(title=f"Final area with more than {MIN_N_VISITS} visits extrapolated from date to end with baseline", x_axis_label='Date', y_axis_label='Metric value', x_axis_type='datetime')
extrapolated_metric_fig.ray(x=fOArea['date'].min(), y=baseline_final_fOArea, length=0, angle=0, width=5, color='lightgreen')
extrapolated_metric_fig.line(x='date', y='extrapolated', color='black', source=fOArea_ds)
extrapolated_metric_fig.scatter(x='date', y='extrapolated', color='black', source=fOArea_ds)

metric_figs = bokeh.layouts.row(metric_at_date_fig, extrapolated_metric_fig)
bokeh.io.show(metric_figs)

## Metric maps

Each subplot represents the map of a metric a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

**Any MAF metric that uses a healpix slicer can be added to this set of maps.**

Annotations are similar to those of the "Visit Map" above:

 - The blue backdrop represents the value of the metric.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.

### Accumulated numbers of visits

In [ ]:
with plt.ioff():
    map_html = "<details><summary><b>Total number of visits accumumlated so far</b></summary>"
    
    fig = map_metric_in_laea_and_mcbryde(
        survey_visits,
        maf.CountMetric(col='band'),
        schedview.plot.survey_skyproj.map_healpix,
        observatory,
        night_events,
        lon_0=night_events.loc['night_middle', 'LST'].item(),
        figsize=(15, 5),
        cmap='cividis_r',
        horizons=None,
    )
    
    map_html = map_html + mpl_fig_to_html(fig)
    map_html = map_html + "</details>"

    for band in 'ugrizy':
        if band not in survey_visits.band.values:
            continue
            
        visits_in_band = survey_visits.loc[survey_visits.band == band, :]
        if len(visits_in_band.loc[~ np.isnan(visits_in_band.eff_time_median.values.astype(float)), 'eff_time_median']) < 1:
            continue

        map_html = map_html + f"""
        <details><summary><b>Total accumulated visits in {band} band</b></summary>
        """

        fig = map_metric_in_laea_and_mcbryde(
            visits_in_band,
            maf.CountMetric(col='band'),
            schedview.plot.survey_skyproj.map_healpix,
            observatory,
            night_events,
            lon_0=night_events.loc['night_middle', 'LST'].item(),
            figsize=(15, 5),
            cmap='cividis_r',
            horizons=None)
        map_html = map_html + mpl_fig_to_html(fig)
        map_html = map_html + "</details>"

    display(HTML(map_html))

### Map of depth (inverse variance of noise) for ideal coadd of visits accumulated so far

The following map shows accumulated inverse variance of completed visits, a uniformly increasing measure of progress toward a target limiting coadd limiting magnitude. The inverse variance of the noise is increases linearly with the number of exposures for exposures of uniform depth, and total exposure time when noise is dominated by sky brightness, and so is sometimes scaled to a time under reference conditions and called the "effective exposure time." See DMTN-296: Calculations of Image and Catalog Depth for more details.


In [ ]:
with plt.ioff():
    map_html = ""
    for band in 'ugrizy':
        if band not in survey_visits.band.values:
            continue
            
        visits_in_band = survey_visits.loc[survey_visits.band == band, :]
        if len(visits_in_band.loc[~ np.isnan(visits_in_band.eff_time_median.values.astype(float)), 'eff_time_median']) < 1:
            continue

        map_html = map_html + f"""
        <details><summary><b>Accumulated inverse noise variance (t<sub>eff</sub>) in {band} band</b></summary>
        """

        fig = map_metric_in_laea_and_mcbryde(
            visits_in_band,
            maf.SumMetric(col='eff_time_median'),
            schedview.plot.survey_skyproj.map_healpix,
            observatory,
            night_events,
            lon_0=night_events.loc['night_middle', 'LST'].item(),
            figsize=(15, 5),
            cmap='cividis_r',
            horizons=None)
        map_html = map_html + mpl_fig_to_html(fig)
        map_html = map_html + "</details>"

    display(HTML(map_html))

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

if len(ddf_visits) > 0:
    nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits, target_column='field_name')
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd, target_column='field_name'
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")


The y-axis (height of the vertical bars) represents the accumulated effective exposure time, $t_{\mbox{eff}}$ (as defined [DMTN-296](https://dmtn-296.lsst.io/)) accumulated over all exposures on the field for the night, colored by filter.

## Hourglass plots

In the hourglass plots that follow, each horizontal band represents a timeline for a day of a month. The gray background shows twilight, while the black background is fully night.
The moon rises or sets where the dotted line crosses each night's horizontal line, and transits where the thick solid line crosses it.

The quantities shown here are just a few examples. Similar hourglass plots can be added for any quantity that varies by visit, and whose value can be mapped to a scalar or category.


In [ ]:
class SingularPassMetric(maf.metrics.PassMetric):
    def run(self, data_slice, slice_point=None):
        # PassMetric returns all columns, but we need just the requested one.
        return data_slice[self.col_name_arr[0]]

In [ ]:
year_months = []
first_date, last_date = [DayObs.from_time(m).date for m in visits.observationStartMJD.describe()[['min', 'max']]]
for dt in range((last_date - first_date).days + 1):
    this_date = first_date + datetime.timedelta(dt)
    year_month = (this_date.year, this_date.month)
    if year_month not in year_months:
        year_months.append(year_month)

In [ ]:
def plot_hourglass_metric(visits, metric, name, constraint="", plot_dict={}, plotter=maf.plots.MonthHourglassPlot):
    with plt.ioff():
        figs_html = ""
        metric_bundle = maf.metric_bundles.MetricBundle(
            metric=metric,
            slicer=maf.slicers.VisitIntervalSlicer(),
            constraint=constraint,
            plot_dict=plot_dict,
        )
        schedview.compute.maf.compute_metric(visits, metric_bundle)
        # If we pass a list of plot_funcs to the MetricBundle, they all
        # get the same plot_dict, an so same title. We want different
        # titles for each month, so iterate here:
        figs = {}
        for year, month in year_months:
            title = f"{name} for {calendar.month_name[month]}, {year}"
            metric_bundle.plot_dict['title'] = title
            metric_bundle.plot_funcs = [plotter(month, year)]
            these_figs = metric_bundle.plot()
            assert len(these_figs) == 1
            this_fig = next(iter(these_figs.values()))
            this_fig_html = mpl_fig_to_html(this_fig)
            figs_html += f"""
                <details>
                <summary><b>Hourglass of {title}</b></summary>
                {this_fig_html}
                </details>
            """

    display(HTML(figs_html))
            

### Seeing

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("seeingFwhmEff"), "seeingFwhmEff")

### Hour Angle

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("HA"), "Hour Angle", plot_dict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5)})

### Band

In [ ]:
plot_dict = {
    "cmap": plt.get_cmap("Set1"),
    "assigned_colors": OrderedDict(
        [("u", 1), ("g", 2), ("r", 4), ("i", 7), ("z", 0), ("y", 3)]
    ),
    "legend_ncols": 1,
    "legend_loc": (1.01, 0.5),
    "legend_bbox_to_anchor": (1.01, 0.0),
    "legend": True,
}

plot_hourglass_metric(visits, SingularPassMetric('band'), "Band", plot_dict=plot_dict, plotter=maf.plots.MonthHourglassCategoricalPlot)